In [102]:
import numpy as np
import pandas as pd
df_train = pd.read_excel('/kaggle/input/training-set/Mix Design Report.xlsx')
# check for missing values
missing_values = df_train.isna().sum()

# print the number of missing values for each column
print(missing_values)

No.                                               0
Date                                              0
Location                                       1017
Type of Coarse Aggregate                          0
Type of Fine Aggregate                            0
Max. Size of Coarse Aggregate (mm)                4
Passing 0.6mm Sieve(%)                          107
Target Mean Strength (N/mm2)                      2
Cement O.P.C (Kg/m3)                              0
W/C Ratio                                         0
Water Content (Kg/m3)                             0
Additive Type & Dos. (/100 Kg of cement)        526
Total Aggregate (Kg/m3)                           0
Fine Aggregate (Kg/m3)                            0
Coarse Aggregate (Kg/m3)                         38
Workability Slump (mm)                            3
Hardened Concrete Desnity (avg.)                  1
Strength (N/mm2)             (7 days avg.)       10
Strength (N/mm2)             (28 days avg.)     666
dtype: int64

In [103]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.regression.linear_model import OLS
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras.backend as K
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt

In [104]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train = df_train.drop(columns=["No.","Date","Strength (N/mm2)             (7 days avg.)","Location","Additive Type & Dos. (/100 Kg of cement)","Total Aggregate (Kg/m3)","W/C Ratio","Hardened Concrete Desnity (avg.)"])

In [105]:
# Define the category to be deleted
category_to_delete = 'n'

# Find the column where you want to delete category
column = 'Max. Size of Coarse Aggregate (mm)'

# Drop the category from the column
df_train[column].replace(category_to_delete, pd.np.nan, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [106]:
# Define the category to be deleted
category_to_delete = 'CALcrete'

# Find the column where you want to delete category
column = 'Target Mean Strength (N/mm2)'

# Drop the category from the column
df_train[column].replace(category_to_delete, pd.np.nan, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [107]:
# Define the category to be deleted
category_to_delete = 40

# Find the column where you want to delete category
column = 'Type of Fine Aggregate'

# Drop the category from the column
df_train[column].replace(category_to_delete, pd.np.nan, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [108]:
# select columns with object data type
object_columns = df_train.select_dtypes(include='object')

# print the unique categories of each column
for column in object_columns:
    print(f"Column: {column}, Categories: {object_columns[column].unique()}")

Column: Type of Coarse Aggregate, Categories: ['Crushed' 'Natural']
Column: Type of Fine Aggregate, Categories: ['Natural' nan 'Crushed']


In [109]:
df_train.dropna(inplace=True)

In [110]:
# get categorical columns
cat_cols = df_train.select_dtypes(include='object').columns

# create an empty dictionary to store the results
results = {}
# loop through each categorical column
for col in cat_cols:
    # get the value counts for the column
    value_counts = df_train[col].value_counts()
    # add the value counts to the results dictionary
    results[col] = value_counts

print(results)

{'Type of Coarse Aggregate': Natural    340
Crushed    156
Name: Type of Coarse Aggregate, dtype: int64, 'Type of Fine Aggregate': Natural    493
Crushed      3
Name: Type of Fine Aggregate, dtype: int64}


In [111]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc = OrdinalEncoder()
df_train[["Type of Coarse Aggregate","Type of Fine Aggregate"]] = enc.fit_transform(df_train[["Type of Coarse Aggregate","Type of Fine Aggregate"]])

In [112]:
# Print all the columns
print(df_train.columns)

Index(['Type of Coarse Aggregate', 'Type of Fine Aggregate',
       'Max. Size of Coarse Aggregate (mm)', 'Passing 0.6mm Sieve(%)',
       'Target Mean Strength (N/mm2)', 'Cement O.P.C (Kg/m3)',
       'Water Content (Kg/m3)', 'Fine Aggregate (Kg/m3)',
       'Coarse Aggregate (Kg/m3)', 'Workability Slump (mm)',
       'Strength (N/mm2)             (28 days avg.)'],
      dtype='object')


In [113]:
print(df_train.head(5))

    Type of Coarse Aggregate  Type of Fine Aggregate  \
0                        0.0                     1.0   
6                        1.0                     1.0   
10                       1.0                     1.0   
12                       0.0                     1.0   
14                       1.0                     1.0   

    Max. Size of Coarse Aggregate (mm)  Passing 0.6mm Sieve(%)  \
0                                 20.0                    56.0   
6                                 20.0                    57.9   
10                                20.0                    31.3   
12                                40.0                    53.9   
14                                40.0                    53.2   

    Target Mean Strength (N/mm2)  Cement O.P.C (Kg/m3)  Water Content (Kg/m3)  \
0                           43.0                   375                    175   
6                           43.0                   365                    165   
10                     

In [129]:
# Split the training data into features and labels
X_train = df_train.drop(columns=['Strength (N/mm2)             (28 days avg.)'])
y_train = df_train['Strength (N/mm2)             (28 days avg.)']
# Define the columns to be scaled
columns_to_exclude = ["Type of Coarse Aggregate","Type of Fine Aggregate"]
columns = [col for col in X_train.select_dtypes(['float64', 'int64']).columns if col not in columns_to_exclude]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data
scaler.fit(X_train[columns])

# Perform min-max scaling on the columns
X_train[columns] = scaler.transform(X_train[columns])
X_train.to_csv('submission_to_submit3.csv', index=False)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)


from statsmodels.stats.stattools import durbin_watson

# Fit a linear regression model to the data
model = OLS(y_train, X_train).fit()

# Perform the Durbin-Watson test
dw_stat = durbin_watson(model.resid)

# Print the Durbin-Watson statistic
print("Durbin-Watson statistic: ", dw_stat)

# Check for autocorrelation
if dw_stat < 2 or dw_stat > 2:
    print("There is autocorrelation in the data, the model is likely non-linear")
else:
    print("There is no autocorrelation in the data, the model is likely linear")
# Define the model
model =KNeighborsRegressor(n_neighbors=5)

# Fit the model to the training data
model.fit(X_train, y_train)


# Make predictions on the validation set
predictions = model.predict(X_val)

# Calculate the mean squared error
mse = mean_squared_error(y_val, predictions)

# Take the square root of the mean squared error to get the root mean squared error
rmse = np.sqrt(mse)
print("RMSE:", rmse)
# Calculate the R-squared score
r2 = r2_score(y_val, predictions)

# Print the R-squared score
print("R-squared:", r2)

Durbin-Watson statistic:  1.903008438896766
There is autocorrelation in the data, the model is likely non-linear
RMSE: 5.353718936553627
R-squared: 0.33302019152061424
